<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/code/RBD_Intersectionality_IAT_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [55]:
import pandas as pd #for dealing with csv import
import os # for joining paths and filenames sensibly
import numpy as np #for the population std
import glob # for finding csv data files
import platform # paths use different dividers on linux vs windows, so we need to test for this
import string

## Define Utility functions

In [56]:
#Define Utility functions
def adjustedmean(RTs,corrs,penalty):
    n=len(corrs) #trials
    n_errors=n-sum(corrs) #errors
    cor_RTs=np.array(corrs)*RTs #sum of correct RTs
    cor_mean=sum(cor_RTs)/sum(corrs)
    
    #mean with errors replaced with penalty value
    return cor_mean+(n_errors*penalty)/n



def exclude_slows(RTs,corrs,slowRT_limit):
    new_rt=[] #holding variables
    new_cr=[]
    for i in range(len(RTs)): #iterate over every item
        if RTs[i] < slowRT_limit: #if it isn't too fast, include RT and corr values
            new_rt.append(RTs[i])
            new_cr.append(corrs[i])
    
    return (new_rt, new_cr)

## Load data

In [57]:
df_IAT_BM = pd.read_csv ('https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/data/IAT_BM.csv', header=0)
df_IAT_WM = pd.read_csv ('https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/data/IAT_WM.csv', header=0)

##Specify thresholds

In [58]:
penalty=0.600 #penalty - in seconds - for incorrect responses
slowRT_limit=10 #threshold at which slow RTs are discarded
fastRT_limit=0.300 #threshold which defines responses which are "too fast"
fast_prop_limit=0.1 # threshold proportion of "too fast" responses which defines exclusion of ppt


## Data extraction

In [71]:
df_IAT_BM.rename(columns={"Spreadsheet: metadata": "Congruence", "allocator-k3xu": "Group Type"}, inplace = True)
df_IAT_BM_2 = df_IAT_BM.drop(["Spreadsheet: ImageLeft", "Spreadsheet: ImageRight"], axis = 1)
df_IAT_WM.rename(columns={"Spreadsheet: metadata": "Congruence", "allocator-k3xu": "Group Type"}, inplace = True)
df_IAT_WM_2 = df_IAT_WM.drop(["Spreadsheet: ImageLeft", "Spreadsheet: ImageRight"], axis = 1)
# df_IAT_BM_2
# df_IAT_WM_2

df_IAT_BM_STR = df_IAT_BM_2[((df_IAT_BM_2["Spreadsheet: TextLeft"] == "Strong") | (df_IAT_BM_2["Spreadsheet: TextRight"] == "Strong")) & (~df_IAT_BM_2["Congruence"].str.startswith("practice"))]
# df_IAT_BM_STR = df_IAT_BM_STR[df_IAT_BM_STR["Congruence"]]
print(df_IAT_BM_STR.shape)
df_IAT_BM_INT = df_IAT_BM_2[((df_IAT_BM_2["Spreadsheet: TextLeft"] == "Intelligence") | (df_IAT_BM_2["Spreadsheet: TextRight"] == "Intelligence")) & (~df_IAT_BM_2["Congruence"].str.startswith("practice"))]
print(df_IAT_BM_INT.shape)
# df_IAT_WM
# #extract the stuff we're interested in (n.b i am indexing using the column names defined in the csv)
#     #dropna() drops nans
#     #tolist() converts from series to list
#     corrs=df_IAT_BM['key_resp_9.corr'].dropna().tolist()
#     rts=df['key_resp_9.rt'].dropna().tolist()
#     block_length=int(len(corrs)/2)
#     #find order 
#     order=df['order'].tolist()[0]
#     #1 congr then incong
#     #2 incongr then congr
    
#     if order==1:
#         congr_corr=corrs[0:block_length]
#         congr_rts=rts[0:block_length]
#         incon_corr=corrs[block_length:]
#         incon_rts=rts[block_length:]
#     else:
#         congr_corr=corrs[block_length:]
#         congr_rts=rts[block_length:]
#         incon_corr=corrs[0:block_length]
#         incon_rts=rts[0:block_length]


(5543, 8)
(5529, 8)


## Reference: 
IAT data analysis script adapted from https://github.com/tomstafford/IAT

Calculate IAT score from data generated by Robin's PsychoPy script [TQS Feb 2014]

*Greenwald, A. G., Nosek, B. A., & Banaji, M. R. (2003). Understanding and using the implicit association test: I. An improved scoring algorithm. Journal of personality and social psychology, 85(2), 1972-216.*